In [ ]:
## IMPORT LIBRARIES ##
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data = pd.read_csv("Combined_data.csv")
data.head()

,text,labels
0,"Hey, a guy I know is breathing down my neck to...",0
1,Mr Deeds! am i 87? these numbers mean nothing ...,0
2,Why nothing. Ok anyway give me treat,0
3,no child support.,0
4,why do you want a massage,1


In [ ]:
data_train = data[:1000]
data_test = data[1000:1500]

In [ ]:
## PREPROCESSING ##
stopwords_en = set(stopwords.words('english'))
def clean_data(data):
  cleaned_text = []
  for sent in data['text']:
    sent = sent.lower()
    sent = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', 
                  '', sent, flags=re.MULTILINE)
    sent = re.sub('[^a-zA-Z]', ' ', sent)
    sent = re.sub(r"won\'t", "will not", sent)
    sent = re.sub(r"can\'t", "can not", sent)
    sent = re.sub(r"n\'t", " not", sent)
    sent = re.sub(r"\'re", " are", sent)
    sent = re.sub(r"\'s", " is", sent)
    sent = re.sub(r"\'d", " would", sent)
    sent = re.sub(r"\'ll", " will", sent)
    sent = re.sub(r"\'t", " not", sent)
    sent = re.sub(r"\'ve", " have", sent)
    sent = re.sub(r"\'m", " am", sent)
    ps = PorterStemmer()
    sent = " ".join([ps.stem(word) for word in sent.split() if word not in stopwords_en and not word.isdigit()])
    cleaned_text.append(sent)
  return cleaned_text

In [ ]:
train_clean = clean_data(data_train)
test_clean = clean_data(data_test)

In [ ]:
train_clean[:5]

['hey guy know breath neck get bud anyway abl get half track usf tonight',
 'mr deed number mean noth fair well mr puffi jacket man ha ha ha',
 'noth ok anyway give treat',
 'child support',
 'want massag']

In [ ]:
## TFDIF VECTOR ##
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=6000)
tfidf_vect_ngram.fit(train_clean)
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_clean)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(test_clean)

In [ ]:
train_y = data_train['labels']
test_y = data_test['labels']

In [ ]:
## SVM ##
model = svm.LinearSVC()
model.fit(xtrain_tfidf_ngram, train_y)
predicted= model.predict(xvalid_tfidf_ngram)
score=accuracy_score(test_y,predicted)
cm = confusion_matrix(test_y,predicted)
print("{}, {}".format(cm,score))

[[ 49 205]
 [ 10 236]], 0.57


In [ ]:
## XGBOOST ##
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(xtrain_tfidf_ngram, train_y)
y_pred = model.predict(xvalid_tfidf_ngram)
score = accuracy_score(test_y, y_pred)
cm = confusion_matrix(test_y,y_pred)
print("{}, {}".format(cm,score))

[[253   1]
 [243   3]], 0.512
